In [14]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
from pathlib import Path
import shutil
import numpy as np
import thor.data as data
import thor.data.dispatch as dispatch
import thor.grid as grid
import thor.track as track
import thor.option as option
import thor.visualize as visualize
import thor.analyze as analyze

notebook_name = "cpol_demo.ipynb"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
# Parent directory for saving outputs
base_local = Path.home() / "THOR_output"
start = "2005-11-13T16:00:00"
end = "2005-11-13T18:00:00"

output_directory = base_local / "runs/cpol_demo_geographic"
options_directory = output_directory / "options"

if output_directory.exists():
    shutil.rmtree(output_directory)

# Create the data_options dictionary
converted_options = {"save": True, "load": False, "parent_converted": None}
cpol_options = data.aura.cpol_data_options(
    start=start, end=end, converted_options=converted_options
)
# Restrict the ERA5 data to a smaller region containing the CPOL radar
lon_range = [129, 133]
lat_range = [-14, -10]
era5_pl_options = data.era5.data_options(
    start=start, end=end, latitude_range=lat_range, longitude_range=lon_range
)
era5_sl_options = data.era5.data_options(
    start=start,
    end=end,
    data_format="single-levels",
    latitude_range=lat_range,
    longitude_range=lon_range,
)
data_options = option.consolidate_options(
    [cpol_options, era5_pl_options, era5_sl_options]
)

dispatch.check_data_options(data_options)
data.option.save_data_options(data_options, options_directory)

altitude = list(np.arange(0, 25e3 + 500, 500))
altitude = [float(alt) for alt in altitude]
grid_options = grid.create_options(name="geographic", altitude=altitude)
grid.check_options(grid_options)
grid.save_grid_options(grid_options, options_directory)

# Create the track_options dictionary
track_options = option.mcs(dataset="cpol")
option.save_track_options(track_options, options_directory)

# Create the display_options dictionary
visualize_options = {
    obj: visualize.option.runtime_options(obj, save=True, style="presentation")
    for obj in ["mcs"]
}
visualize_options["middle_echo"] = visualize.option.runtime_options(
    "middle_echo", save=True, style="presentation", figure_types=["mask"]
)
# visualize.option.save_display_options(visualize_options, options_directory)
visualize_options = None

2024-10-03 18:55:32,776 - thor.option - DEBUG - Saving options to /home/ewan/THOR_output/runs/cpol_demo_geographic/options/data.yml
2024-10-03 18:55:32,797 - thor.grid - WARNING - Shape not specified. Will attempt to infer from input.
2024-10-03 18:55:32,802 - thor.grid - WARNING - Coordinates not specified. Will attempt to infer from input.
2024-10-03 18:55:32,807 - thor.option - DEBUG - Saving options to /home/ewan/THOR_output/runs/cpol_demo_geographic/options/grid.yml
2024-10-03 18:55:32,839 - thor.option - DEBUG - Saving options to /home/ewan/THOR_output/runs/cpol_demo_geographic/options/track.yml


In [16]:
times = data.utils.generate_times(data_options["cpol"])
tracks = track.simultaneous_track(
    times,
    data_options,
    grid_options,
    track_options,
    visualize_options,
    output_directory=output_directory,
    parallel=False
)

2024-10-03 18:55:41,681 - thor.track - INFO - Beginning thor run. Saving output to /home/ewan/THOR_output/runs/cpol_demo_geographic.
2024-10-03 18:55:41,683 - thor.track - INFO - Beginning simultaneous tracking.


2024-10-03 18:55:41,743 - thor.track - INFO - Processing 2005-11-13T16:00:08.
2024-10-03 18:55:41,745 - thor.data.aura - INFO - Updating cpol dataset for 2005-11-13T16:00:08.
2024-10-03 18:55:41,745 - thor.data.aura - INFO - Converting cpol data from twp10cpolgrid150.b2.20051113.160000.nc
2024-10-03 18:55:41,789 - thor.data.aura - INFO - Creating new geographic grid with spacing 0.025 m, 0.025 m.
2024-10-03 18:55:42,338 - thor.track - INFO - Processing hierarchy level 0.
2024-10-03 18:55:42,339 - thor.track - INFO - Tracking cell.
2024-10-03 18:55:42,362 - thor.match.match - INFO - Matching cell objects.
2024-10-03 18:55:42,363 - thor.match.match - INFO - No previous mask, or no objects in previous mask.
2024-10-03 18:55:42,368 - thor.visualize.runtime - INFO - Generating runtime visualizations.
2024-10-03 18:55:42,370 - thor.track - INFO - Tracking middle_echo.
2024-10-03 18:55:42,381 - thor.visualize.runtime - INFO - Generating runtime visualizations.
2024-10-03 18:55:42,382 - thor.t

# Cartesian Coordinates

In [22]:
output_directory = base_local / "runs/cpol_demo_cartesian"
options_directory = output_directory / "options"

if output_directory.exists():
    shutil.rmtree(output_directory)

grid_options = grid.create_options(name="cartesian", regrid=False, altitude=altitude)
grid.check_options(grid_options)
grid.save_grid_options(grid_options, options_directory)
data.option.save_data_options(data_options, options_directory)
option.save_track_options(track_options, options_directory)
# visualize.option.save_display_options(visualize_options, options_directory)

times = data.utils.generate_times(data_options["cpol"])
tracks = track.simultaneous_track(
    times,
    data_options,
    grid_options,
    track_options,
    visualize_options,
    output_directory=output_directory,
)

2024-10-03 13:55:34,330 - thor.grid - WARNING - Shape not specified. Will attempt to infer from input.
2024-10-03 13:55:34,333 - thor.option - DEBUG - Saving options to /home/ewan/THOR_output/runs/cpol_demo_cartesian/options/grid.yml
2024-10-03 13:55:34,349 - thor.option - DEBUG - Saving options to /home/ewan/THOR_output/runs/cpol_demo_cartesian/options/data.yml
2024-10-03 13:55:34,406 - thor.option - DEBUG - Saving options to /home/ewan/THOR_output/runs/cpol_demo_cartesian/options/track.yml


# Analysis

In [4]:
analysis_options = analyze.mcs.analysis_options()
analyze.mcs.process_velocities(output_directory)
analyze.mcs.quality_control(output_directory, analysis_options)
analyze.mcs.classify_all(output_directory)

2024-10-03 16:11:10,765 - thor.write.attribute - DEBUG - Writing attribute dataframe to /home/ewan/THOR_output/runs/cpol_demo_geographic/analysis/velocities.csv
2024-10-03 16:11:10,768 - thor.write.attribute - DEBUG - Saving attribute metadata to /home/ewan/THOR_output/runs/cpol_demo_geographic/analysis/velocities.yml
2024-10-03 16:11:10,900 - thor.write.attribute - DEBUG - Writing attribute dataframe to /home/ewan/THOR_output/runs/cpol_demo_geographic/analysis/quality.csv
2024-10-03 16:11:10,903 - thor.write.attribute - DEBUG - Saving attribute metadata to /home/ewan/THOR_output/runs/cpol_demo_geographic/analysis/quality.yml
2024-10-03 16:11:10,962 - thor.write.attribute - DEBUG - Writing attribute dataframe to /home/ewan/THOR_output/runs/cpol_demo_geographic/analysis/classification.csv
2024-10-03 16:11:10,964 - thor.write.attribute - DEBUG - Saving attribute metadata to /home/ewan/THOR_output/runs/cpol_demo_geographic/analysis/classification.yml


,,stratiform_offset,inflow,relative_stratiform_offset,tilt,propagation
time,universal_id,,,,,
2005-11-13 16:00:08,1,trailing,front,trailing,up-shear,down-shear
2005-11-13 16:10:23,1,trailing,front,trailing,up-shear,down-shear
2005-11-13 16:20:09,1,trailing,front,trailing,up-shear,down-shear
2005-11-13 16:30:09,1,trailing,front,trailing,up-shear,down-shear
2005-11-13 16:40:09,1,trailing,front,trailing,up-shear,down-shear
2005-11-13 16:50:08,1,trailing,front,trailing,up-shear,down-shear
2005-11-13 17:00:08,1,trailing,front,trailing,up-shear,down-shear
2005-11-13 17:10:23,1,trailing,front,trailing,up-shear,down-shear
2005-11-13 17:20:09,1,trailing,front,trailing,up-shear,down-shear


In [7]:
figure_options = visualize.option.horizontal_attribute_options(
    "mcs_velocity_analysis", style="paper"
)
start_time = np.datetime64("2005-11-13T16:00")
end_time = np.datetime64("2005-11-13T19:00")
visualize.attribute.mcs_series(output_directory, start_time, end_time, figure_options)

2024-10-03 16:20:09,966 - thor.data.aura - INFO - Converting cpol data from twp10cpolgrid150.b2.20051113.160000.nc
2024-10-03 16:20:10,011 - thor.data.aura - INFO - Creating new geographic grid with spacing 0.025 m, 0.025 m.
2024-10-03 16:20:10,463 - thor.visualize.horizontal - INFO - No figsize provided. Using default.
2024-10-03 16:20:10,565 - thor.visualize.horizontal - DEBUG - Plotting boundary.
2024-10-03 16:20:10,583 - thor.visualize.horizontal - DEBUG - Plotting boundary.
2024-10-03 16:20:10,677 - thor.visualize.attribute - DEBUG - Saving mcs_velocity_analysis figure for 2005-11-13T16:00:08.000000000.
2024-10-03 16:20:12,125 - thor.data.aura - INFO - Converting cpol data from twp10cpolgrid150.b2.20051113.161000.nc
2024-10-03 16:20:12,903 - thor.visualize.horizontal - INFO - No figsize provided. Using default.
2024-10-03 16:20:13,001 - thor.visualize.horizontal - DEBUG - Plotting boundary.
2024-10-03 16:20:13,056 - thor.visualize.horizontal - DEBUG - Plotting boundary.
2024-10-03